# Imports

In [1]:
import datetime
import json
from time import time
import os
from collections import defaultdict
import pickle
from tqdm import tqdm

# from functools import partial
# from itertools import chain, islice
# from multiprocessing import Pool, cpu_count
from pathlib import Path

# from pprint import pprint

import dateutil.parser
import findspark
import lmdb
import numpy as np
import psycopg
import pyspark.sql.functions as sf
from ase.atoms import Atoms
from ase.io.cfg import read_cfg
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    ArrayType,
    BooleanType,
    DoubleType,
    FloatType,
    IntegerType,
    LongType,
    StringType,
    StructField,
    StructType,
    TimestampType,
)
from colabfit.tools.schema import (
    property_object_schema,
    config_df_schema,
    config_schema,
    property_object_df_schema,
)
from colabfit.tools.configuration import AtomicConfiguration, config_schema
from colabfit.tools.database import DataManager, PGDataLoader
from colabfit.tools.dataset import Dataset, dataset_schema
from colabfit.tools.property import Property, property_object_schema
from colabfit.tools.property_definitions import (
    atomic_forces_pd,
    cauchy_stress_pd,
    potential_energy_pd,
)
from colabfit.tools.schema import configuration_set_schema
import pyarrow as pa

with open("formation_energy.json", "r") as f:
    formation_energy_pd = json.load(f)
findspark.init()
format = "jdbc"
load_dotenv("./.env")

True

In [ ]:
import colabfit.tools.configuration
from importlib import reload

reload(colabfit.tools.configuration)
AtomicConfiguration = colabfit.tools.configuration.AtomicConfiguration

# Connect to DB and run loader

In [23]:
JARFILE = os.environ.get("CLASSPATH")
spark = (
    SparkSession.builder.appName("PostgreSQL Connection with PySpark")
    .config("spark.jars", JARFILE)
    .getOrCreate()
)
url = "jdbc:postgresql://localhost:5432/colabfit"
user = os.environ.get("PGS_USER")
password = os.environ.get("PGS_PASS")
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver",
}
loader = PGDataLoader(appname="colabfit", env="./.env")

24/05/21 14:49:53 WARN Utils: Your hostname, arktos resolves to a loopback address: 127.0.1.1; using 172.24.21.25 instead (on interface enp5s0)
24/05/21 14:49:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/05/21 14:49:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/21 14:49:55 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
import os
import pickle
import sys
import time
from pathlib import Path

from ase.io import iread
from dotenv import load_dotenv
from tqdm import tqdm

from colabfit.tools.configuration import AtomicConfiguration
from colabfit.tools.database import DataManager, SparkDataLoader
from colabfit.tools.property_definitions import (
    atomic_forces_pd,
    free_energy_pd,
    potential_energy_pd,
)

load_dotenv()
loader = SparkDataLoader(table_prefix="ndb.colabfit.dev")
access_key = os.getenv("SPARK_ID")
access_secret = os.getenv("SPARK_KEY")
endpoint = os.getenv("SPARK_ENDPOINT")
# loader.set_vastdb_session(
PKL_FP = Path("data/oc20_data_mapping.pkl")
with open(PKL_FP, "rb") as f:
    OC20_MAP = pickle.load(f)

In [2]:
from colabfit.tools.schema import *

In [7]:
schema[40:60]

StructType([StructField('atomic_forces_11', StringType(), True), StructField('atomic_forces_12', StringType(), True), StructField('atomic_forces_13', StringType(), True), StructField('atomic_forces_14', StringType(), True), StructField('atomic_forces_15', StringType(), True), StructField('atomic_forces_16', StringType(), True), StructField('atomic_forces_17', StringType(), True), StructField('atomic_forces_18', StringType(), True), StructField('atomic_forces_19', StringType(), True), StructField('atomic_forces_unit', StringType(), True), StructField('atomic_forces_property_id', StringType(), True), StructField('cauchy_stress', StringType(), True), StructField('cauchy_stress_unit', StringType(), True), StructField('cauchy_stress_volume_normalized', BooleanType(), True), StructField('cauchy_stress_property_id', StringType(), True), StructField('electronic_free_energy', DoubleType(), True), StructField('electronic_free_energy_unit', StringType(), True), StructField('electronic_free_energy

In [8]:
schema[41].dataType.typeName()

'string'

In [9]:
schema = property_object_df_schema
string_cols = [f.name for f in schema if f.dataType.typeName() == "array"]
print(string_cols)

['atomic_forces_00', 'atomic_forces_01', 'atomic_forces_02', 'atomic_forces_03', 'atomic_forces_04', 'atomic_forces_05', 'atomic_forces_06', 'atomic_forces_07', 'atomic_forces_08', 'atomic_forces_09', 'atomic_forces_10', 'atomic_forces_11', 'atomic_forces_12', 'atomic_forces_13', 'atomic_forces_14', 'atomic_forces_15', 'atomic_forces_16', 'atomic_forces_17', 'atomic_forces_18', 'atomic_forces_19', 'cauchy_stress']


In [ ]:
from importlib import reload

import colabfit.tools.utilities
import colabfit.tools.dataset
import colabfit.tools.database
import colabfit.tools.configuration_set
import colabfit.tools.property

reload(colabfit.tools.utilities)
reload(colabfit.tools.dataset)
reload(colabfit.tools.database)
reload(colabfit.tools.property)
DataManager = colabfit.tools.database.DataManager
ConfigurationSet = colabfit.tools.configuration_set.ConfigurationSet
Dataset = colabfit.tools.dataset.Dataset
Property = colabfit.tools.property.Property

df.rdd  
you can only parallelize one time so don't try to do a dataframe select from an rdd  
updating to sdk 5.1 in a couple weeks  
boto3 and s3 are the amazon file system interactions, mostly for adding metadata TO FILES (not to the database) and interacting with the files as FileExistsError. 
Make sure to spark.stop() at end of  python file.

In [29]:
def update_co_rows_cs_id(self, co_ids: list[str], cs_id: str):
    with psycopg.connect(
        """dbname=colabfit user=%s password=%s host=localhost port=5432"""
        % (
            user,
            password,
        )
    ) as conn:
        # dbname=self.database_name,
        # user=self.properties["user"],
        # password=self.properties["password"],
        # host="localhost",
        # port="5432",
        cur = conn.execute(
            """UPDATE configurations
                SET configuration_set_ids = 
            """
        )
        cur = conn.execute(
            """UPDATE configurations
                SET configuration_set_ids = concat(%s::text, 
                rtrim(ltrim(replace(configuration_set_ids,%s,''), 
                
                '['),']') || ', ', %s::text)
            WHERE id = ANY(%s)""",
            ("[", f"{cs_id}", f"{cs_id}]", co_ids),
            # ("[", f", {cs_id}", f", {cs_id}]"),
        )
        # cur.fetchall()
        conn.commit()

In [ ]:
# You were trying to get  postgresql to recognize the WHERE id = ANY() array syntax

In [112]:
with psycopg.connect(
    dbname="colabfit",
    user=os.environ.get("PGS_USER"),
    password=os.environ.get("PGS_PASS"),
    host="localhost",
) as conn:
    with conn.cursor() as cur:

        # cur.execute(
        #     "UPDATE configurations SET configuration_set_ids = configuration_set_ids || %(cs_id)s WHERE id = ANY(%(co_ids)s)",
        #     {"cs_id": cs["id"], "co_ids": co_ids},
        # )
        # data = cur.fetchall()
        cur.execute(
            "SELECT * FROM public.configurations WHERE id = ANY(%s)",
            [co_ids],
        )
        data2 = cur.fetchall()
    conn.commit()

# In progress

Upsert appears to be this for postgres:
```
update the_table
    set id = id || array[5,6]
where id = 4;
```
* ~~Check for upsert function from pyspark to concatenate lists of relationships instead of primary key id collision~~
* There is no pyspark-upsert function. Will have to manage this possibly through a different sql-based library
* Written: find duplicates, but convert to access database, not download full dataframe
* I see this being used with batches of hashes during upload: something like
    ``` for batch in batches:
            hash_duplicates = find_duplicates(batch, loader/database)
            hash_duplicates.make_change_to_append_dataset-ids
            hash_duplicates.write-to-database
* Where would be the best place to catch duplicates? Keeping in mind that this might be a bulk operation (i.e. on the order of millions, like with ANI1/ANI2x variations)

In [9]:
JARFILE = os.environ.get("CLASSPATH")
spark = (
    SparkSession.builder.appName("PostgreSQL Connection with PySpark")
    .config("spark.jars", JARFILE)
    .getOrCreate()
)
url = "jdbc:postgresql://localhost:5432/colabfit"
user = os.environ.get("PGS_USER")
password = os.environ.get("PGS_PASS")
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver",
}
loader = PGDataLoader(appname="colabfit", env="./.env")

24/05/30 09:52:06 WARN Utils: Your hostname, arktos resolves to a loopback address: 127.0.1.1; using 172.24.21.25 instead (on interface enp5s0)
24/05/30 09:52:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/05/30 09:52:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/30 09:52:08 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
mtpu_ds_id = "DS_y7nrdsjtuwom_0"
mtpu_configs = mtpu_reader(Path("data/mtpu_2023/Unified_training_set.cfg"))
dm2 = DataManager(
    nprocs=4,
    configs=mtpu_configs,
    prop_defs=[potential_energy_pd, atomic_forces_pd, cauchy_stress_pd],
    prop_map=PROPERTY_MAP,
    dataset_id=mtpu_ds_id,
)

In [ ]:
from pyspark.sql import Row


def write_value_to_file(path_prefix, extension, BUCKET_DIR, write_column, row):
    """i.e.: partial(_write_value(
    'CO/positions',
    'txt',
    '/save/here'
    'positions',
    )
    """
    id = row["id"]
    value = row[write_column]
    row_dict = row.copy()
    split = id[-4:]
    filename = f"{id}.{extension}"
    full_path = Path(BUCKET_DIR) / path_prefix / split / filename
    full_path.parent.mkdir(parents=True, exist_ok=True)
    full_path.write_text(str(value))
    # row_dict = row.asDict()
    row_dict[write_column] = str(full_path)
    return Row(**row_dict)


from functools import partial

part_write = partial(
    write_value_to_file,
    "CO/positions",
    "txt",
    "/scratch/gw2338/vast/data-lake-main/spark/scripts",
    "positions",
)

In [ ]:
configs = mtpu_reader(Path("data/mtpu_2023/Unified_training_set.cfg"))
co_rows = [x.spark_row for x in configs]
rdd = sc.parallelize(co_rows)
rdd.foreachPartition(part_write)

In [ ]:
config_list = list(mtpu_reader(Path("data/mtpu_2023/Unified_training_set.cfg")))
dm2.configs = config_list[:50]
dm2.load_co_po_to_vastdb(loader)
dm2.configs = config_list[25:]
dm2.load_co_po_to_vastdb(loader)

In [ ]:
from importlib import reload

import colabfit.tools.utilities
import colabfit.tools.dataset
import colabfit.tools.database
import colabfit.tools.configuration_set
import colabfit.tools.schema

reload(colabfit.tools.utilities)
reload(colabfit.tools.schema)
reload(colabfit.tools.dataset)
reload(colabfit.tools.database)
DataManager = colabfit.tools.database.DataManager
ConfigurationSet = colabfit.tools.configuration_set.ConfigurationSet
Dataset = colabfit.tools.dataset.Dataset
property_object_df_schema = colabfit.tools.schema.property_object_df_schema
property_object_schema = colabfit.tools.schema.property_object_schema
##############

import json
import lmdb
import pickle
from colabfit.tools.database import DataManager, SparkDataLoader

loader = SparkDataLoader(table_prefix="ndb.colabfit.dev")
load_dotenv()
access_key = os.getenv("SPARK_ID")
access_secret = os.getenv("SPARK_KEY")
endpoint = os.getenv("SPARK_ENDPOINT")
loader.set_vastdb_session(
    endpoint=endpoint, access_key=access_key, access_secret=access_secret
)

with open("formation_energy.json", "r") as f:
    formation_energy_pd = json.load(f)

carmat_config_gen = carmat_reader(Path("data/carolina_matdb/base/all/data.mdb"))
carmat_ds_id = "DS_y7nrdsjtuw0g_0"


dm = DataManager(
    nprocs=1,
    configs=carmat_config_gen,
    prop_defs=[formation_energy_pd],
    prop_map=CM_PROPERTY_MAP,
    dataset_id=carmat_ds_id,
)
dm.configs = carmat_reader(Path("data/carolina_matdb/base/all/data.mdb"))

match = [
    (r".*3.*", None, "3_configurations", "Carmat with 3"),
    (r".*4.*", None, "4_configurations", "Carmat with 4"),
]
# dm.load_co_po_to_vastdb(loader)

In [ ]:
from importlib import reload

import colabfit.tools.utilities
import colabfit.tools.dataset
import colabfit.tools.database
import colabfit.tools.configuration
import colabfit.tools.configuration_set
import colabfit.tools.property

reload(colabfit.tools.utilities)
reload(colabfit.tools.dataset)
reload(colabfit.tools.configuration
reload(colabfit.tools.database)
reload(colabfit.tools.property)
Configuration = colabfit.tools.configuration.Configuration
DataManager = colabfit.tools.database.DataManager
ConfigurationSet = colabfit.tools.configuration_set.ConfigurationSet
Dataset = colabfit.tools.dataset.Dataset
Property = colabfit.tools.property.Property

#################################################

batches = dm.gather_co_po_in_batches()
batch = next(batches)
cos, pos = zip(*batch)
from colabfit.tools.schema import *
import pyspark.sql.functions as sf

podf = spark.createDataFrame(pos, schema=property_object_df_schema).limit(100)
from colabfit.tools.utilities import stringify_df_val

schema = property_object_df_schema
string_cols = [f.name for f in schema if f.dataType.typeName() == "array"]
string_col_udf = sf.udf(stringify_df_val, StringType())
for col in string_cols:
    podf = podf.withColumn(col, string_col_udf(sf.col(col)))
podf2 = split_long_string(podf)

codf = spark.createDataFrame(cos, schema=config_df_schema)
combindf = codf.withColumnRenamed("id", "configuration_id").join(
    podf, on="configuration_id", how="inner"
)
combindf.select("id", "configuration_id", "nsites", "atomic_forces_00").limit(1).show()

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
from pyspark.sql.types import StringType

NSITES_COL_SPLITS = 20
from pyspark.sql import DataFrame
from pyspark.sql.functions import substring, length, col, lit

In [ ]:
# def split_long_string(
#     df, column_name: str = "atomic_forces_00", max_string_length: int = 100
# ):
"""
Splits a long string column into multiple columns based on a maximum string length.
:param df: Input DataFrame
:param column_name: Name of the column containing the long string
:param max_string_length: Maximum length for each split string
:return: DataFrame with the long string split across multiple columns
"""
overflow_columns = [
    f"{'_'.join(column_name.split('_')[:-1])}_{i+1:02}" for i in range(19)
]
all_columns = [column_name] + overflow_columns
tmp_columns = [f"{col_name}_tmp" for col_name in all_columns]
df = df.withColumn("total_length", sf.length(sf.col(column_name)))
substring_exprs = [
    sf.when(
        sf.length(sf.col(column_name)) - (i * max_string_length) > 0,
        sf.substring(
            sf.col(column_name), (i * max_string_length + 1), max_string_length
        ),
    )
    .otherwise(sf.lit(None))
    .alias(col_name)
    for i, col_name in enumerate(tmp_columns)
]

substring_exprs = [
    sf.substring(
        df[column_name],
        (i * max_string_length + 1),
        max_string_length,
    ).alias(col_name)
    for i, col_name in enumerate(tmp_columns)
]
df1 = df.select("*", *substring_exprs)
for tmp_col, col in zip(tmp_columns, all_columns):
    df1 = df1.drop(col).withColumnRenamed(f"{tmp_col}", col)
df = df.drop("total_length")
# return df


# Make this to replace the columns, not just add duplicate names
def split_long_string(df, col_name, thresh):
    columns = [sf.col(c) for c in df.columns]
    num_splits = NSITES_COL_SPLITS
    split_exprs = [
        sf.when(
            sf.col(col_name).substr(i * thresh + 1, thresh) != "",
            sf.col(col_name).substr(i * thresh + 1, thresh),
        ).otherwise(sf.lit(""))
        for i in range(num_splits)
    ]
    for i, expr in enumerate(split_exprs):
        columns.append(expr.alias(f"{'_'.join(col_name.split('_')[:-1])}_{i:02}"))
    return df.select(columns)


df1 = split_long_string(podf, "atomic_forces_00", 500 // 19)

# Initialize a Spark session
spark = SparkSession.builder.appName("split_long_string").getOrCreate()

# Sample data
data = [
    (1, "a" * 150),  # Long string
    (2, "short"),  # Short string
    (3, "b" * 250),  # Longer string
]

# Create DataFrame
df = spark.createDataFrame(data, ["id", "stringcol_00"])

# Apply the split_long_string function
result_df = split_long_string(df, "stringcol_00", 100, 3)

# Show the result
result_df.show(truncate=False)

In [34]:
batches = dm.gather_co_po_in_batches()
batch = next(batches)
cos, pos = zip(*batch)
po_rdd = sc.parallelize(pos[:100])
po_rdd2 = check_size_n_arrs_and_split(po_rdd, "atomic_forces")
keys = list(po_rdd2.map(lambda x: x.keys()).take(1)[0])
schema = property_object_schema
extra_keys = [x for x in keys if x not in schema.fieldNames()]
for key in extra_keys:
    schema.add(StructField(key, StringType(), True))

In [ ]:
spark.sql("show tables in ndb.colabfit.dev").show()
spark.sql("drop table ndb.colabfit.dev.sample_configs")
spark.sql("drop table ndb.colabfit.dev.sample_prop_objects")
spark.sql("drop table ndb.colabfit.dev.sample_config_sets")

In [ ]:
from colabfit.tools.utilities import *
from colabfit.tools.database import *
from colabfit.tools.schema import *
from functools import partial

self = loader
batches = dm.gather_co_po_in_batches()
batch = next(batches)

co_rows, po_rows = list(zip(*batch))
co_rdd = loader.spark.sparkContext.parallelize(co_rows)
po_rdd = loader.spark.sparkContext.parallelize(po_rows)
co_ids = co_rdd.map(lambda x: x["id"]).collect()
co_rdd2 = (
    co_rdd.map(lambda x: (x["id"], x)).reduceByKey(lambda a, b: a).map(lambda x: x[1])
)


po_ids = po_rdd.map(lambda x: x["id"]).collect()
if len(set(po_ids)) < len(po_ids):
    print(f"{len(po_ids) - len(set(po_ids))} duplicates found in PO RDD")
    po_rdd = loader.reduce_po_rdd(po_rdd)
co_ids = set(co_ids)
all_unique_co = loader.check_unique_ids(loader.config_table, co_ids)
all_unique_po = loader.check_unique_ids(loader.prop_object_table, po_ids)

new_po_ids, update_po_ids = loader.find_existing_po_rows_append_elem(
    po_rdd=po_rdd,
    ids=po_ids,
)
loader.write_table(
    po_rdd,
    loader.prop_object_table,
    property_object_schema,
    ids_filter=new_po_ids,
)

cos = list(cos)
cos.extend([cos[0], cos[1]])
co_rdd = sc.parallelize(cos)

In [ ]:
from collections import Counter

idc = Counter(ids)
ids = prop_df.select("configuration_id").collect()
ids = [x["configuration_id"] for x in ids]


ids = [x["configuration_id"] for x in ids]
co_po_df.filter(sf.col("configuration_id") == "CO_1077499488010994550").select(
    "potential_energy", "potential_energy_reference", "atomic_forces", "cauchy_stress"
).show()

In [33]:
import numpy as np
from functools import partial


def generate_random_float_array(n):
    # Generate an n*3 array with random floats
    random_array = np.random.rand(n, 3)
    return random_array


def split_string(s, max_length=60000):
    if s is None:
        return [None]
    return [s[i : i + max_length] for i in range(0, len(s), max_length)]


np.set_printoptions(threshold=np.inf)
x = generate_random_float_array(10000)
xstr = "".join(np.array2string(np.arr(x), separator=",").replace("\n", ""))
splx = split_string(xstr)

In [ ]:
from colabfit.tools.utilities import _write_value
from functools import partial

In [ ]:
part_write = partial(
    _write_value,
    "CO/positions",
    "txt",
    "/scratch/gw2338/vast/data-lake-main/spark/scripts",
    "positions",
)

In [ ]:
from importlib import reload

import colabfit.tools.dataset
import colabfit.tools.database
import colabfit.tools.configuration_set
import colabfit.tools.schema

reload(colabfit.tools.configuration_set)
reload(colabfit.tools.dataset)
reload(colabfit.tools.database)
reload(colabfit.tools.schema)
configuration_set_schema = colabfit.tools.schema.configuration_set_schema
DataManager = colabfit.tools.database.DataManager
ConfigurationSet = colabfit.tools.configuration_set.ConfigurationSet
Dataset = colabfit.tools.dataset.Dataset